In [11]:
import plotly.express as px
import pandas as pd
from datatypes import *
from utils import *

# Example DataFrame
df = read_obc(os.path.join(os.getcwd(), "..", "ros2_ws", "src", "simulation", "worlds", "autogen.obc"))

# Create interactive scatter plot
fig = px.scatter_3d(
    df,
    x="x",
    y="y",
    z="z",
    hover_name="id",  # Show label on hover
    title="autogen.obc",
)
fig.update_layout(
    scene=dict(
        aspectmode="data"  # Ensures x, y, z use the same scale
    )
)
fig.update_traces(marker=dict(size=2))
fig.update_layout(
    width=800,   # default ~700
    height=600  # increase vertical size
)
fig.show()
